In [4]:
import lasagne
def build_yolo_classification_cnn(input_var,
                                  num_filters,
                        num_fc_units,
                        num_extra_conv, 
                        num_pool,
                        nonlinearity,
                        w_init,
                        dropout_p
                       ):
    # 8x8x96
    network = lasagne.layers.InputLayer(shape=(None, 8, 96, 96), input_var=input_var)
    
    #64 x 96 x 96 
    network = lasagne.layers.Conv2DLayer(network, num_filters=num_filters / 2., 
                                     filter_size=(3,3),
                                     pad=1,
                                     nonlinearity=nonlinearity,
                                     W=w_init)
    
    for i in range(num_pool - 1):

        #num_filters x 96 / (2*(i+1)) x 96 / (2*(i+1))
        network = lasagne.layers.MaxPool2DLayer(network, 
                                                pool_size=(2,2))

        # num_filters x 96 / (2*(i+1)) x 96 / (2*(i+1))
        network = lasagne.layers.Conv2DLayer(network, num_filters=num_filters, 
                                             filter_size=(3,3),
                                             pad=1,
                                             nonlinearity=nonlinearity,
                                             W=w_init)

    
    #1024 x 96 / (2^num_pool) x 96 / (2^num_pool)
    #average pooling
    network = lasagne.layers.Pool2DLayer(network, pool_size=(2,2), mode='average_exc_pad')                             
    
    network = lasagne.layers.DenseLayer(
                                lasagne.layers.dropout(network, p=dropout_p),
                                num_units=num_fc_units,
                                nonlinearity=lasagne.nonlinearities.rectify)
    
    network = lasagne.layers.DenseLayer(network, 
                                        num_units=2,
                                        nonlinearity=lasagne.nonlinearities.softmax)
    
    return network

In [6]:
def build_network(learning_rate = 0.01,
                  momentum = 0.9,
                  num_filters=128,
                  num_fc_units=1024,
                  num_extra_conv=0, 
                  num_pool=4,
                  nonlinearity=lasagne.nonlinearities.LeakyRectify(0.1),
                  w_init=lasagne.init.HeNormal(),
                  dropout_p=0.5,
                  load=False):
    
    input_var = T.tensor4('inputs')
    target_var = T.ivector('targets')
    print("Building model and compiling functions...")
    network = build_yolo_classification_cnn(input_var)
    if load:
        with np.load('model.npz') as f:
            param_values = [f['arr_%d' % i] for i in range(len(f.files))]
            lasagne.layers.set_all_param_values(network, param_values)

    # Create a loss expression for training, i.e., a scalar objective we want
    # to minimize (for our multi-class problem, it is the cross-entropy loss):
    prediction = lasagne.layers.get_output(network, deterministic=False)
    loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
    loss = loss.mean()

    # Create update expressions for training, i.e., how to modify the
    # parameters at each training step
    params = lasange.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate=learning_rate, momentum=momentum)

    # Create a loss expression for validation/testing. The crucial difference
    # here is that we do a deterministic forward pass through the network,
    # disabling dropout layers.
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                                target_var)
    test_loss = test_loss.mean()



    # As a bonus, also create an expression for the classification accuracy:
    test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                          dtype=theano.config.floatX)

    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_fn = theano.function([input_var, target_var], loss, updates=updates)


    # Compile a second function computing the validation loss and accuracy:
    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

    return train_fn, val_fn, network

In [ ]:
#old
    # 192 x 24 x 24
#     network = lasagne.layers.MaxPool2DLayer(network, 
#                                             pool_size=(2,2))
    
    
    
#     # 128 x 24 x 24
#     network = lasagne.layers.Conv2DLayer(network, num_filters=num_filters, 
#                                          filter_size=(1,1),
#                                          nonlinearity=nonlinearity,
#                                          W=w_init)
    
# #     # 256 x 24 x 24
# #     network = lasagne.layers.Conv2DLayer(network, num_filters=256, 
# #                                          filter_size=(3,3),
# #                                          pad=1,
# #                                          nonlinearity=nonlinearity,
# #                                          W=w_init)
    
# #     # 256 x 24 x 24
# #     network = lasagne.layers.Conv2DLayer(network, num_filters=256, 
# #                                          filter_size=(1,1),
# #                                          nonlinearity=nonlinearity,
# #                                          W=w_init)
    
# #     # 512 x 24 x 24
# #     network = lasagne.layers.Conv2DLayer(network, num_filters=512, 
# #                                          filter_size=(3,3),
# #                                          pad=1, 
# #                                          nonlinearity=nonlinearity,
# #                                          W=w_init)
    
#     #512 x 12 x 12
#     network = lasagne.layers.MaxPool2DLayer(network, 
#                                             pool_size=(2,2))
    
    
#     # 256 x 12 x 12
#     network = lasagne.layers.Conv2DLayer(network, num_filters=num_filters, 
#                                          filter_size=(1,1),
#                                          nonlinearity=nonlinearity,
#                                          W=w_init)
    
# #     # 512 x 12 x 12
# #     network = lasagne.layers.Conv2DLayer(network, num_filters=512, 
# #                                          filter_size=(3,3),
# #                                          pad=1,
# #                                          nonlinearity=nonlinearity,
# #                                          W=w_init)
    


# #     #512 x 12 x 12
# #     network = lasagne.layers.Conv2DLayer(network, num_filters=512, 
# #                                      filter_size=(1,1),
# #                                      nonlinearity=nonlinearity,
# #                                      W=w_init)

# #     #1024 x 12 x 12
# #     network = lasagne.layers.Conv2DLayer(network, num_filters=1024, 
# #                                      filter_size=(3,3),
# #                                      pad=1,
# #                                      nonlinearity=nonlinearity,
# #                                      W=w_init)